# Image Denoising Challenge

The goal for this challenge is to leverage your knowledge of Deep Learning to design and train a denoising model. For a given noisy image $X$, our model should learn to predict the denoised image $y$.


**Objectives**
- Visualize images
- Preprocess images for the neural network
- Fit a custom CNN for the task

## 1. Load Data

👉 Let's download the dataset archive.
It contains RGB and Black & White images we will be using for the rest of this challenge.

In [ ]:
! rm -rf paintings/
! curl https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/paintings.zip > paintings.zip
! LANG=en_US.UTF-8 unzip -nq "paintings.zip" 
! rm "paintings.zip"
! ls -l

In [ ]:
import glob

dataset_paths = glob.glob("./paintings/*.jpg")
print(f"The dataset contains {len(dataset_paths)} images")

❓ **Display the image at index `53` of this dataset_paths (i.e the 54-th image)**

<details>
    <summary>Hint</summary>
    Use the <code>PIL.Image.open</code> and <code>matplotlib.pyplot.imshow</code> functions.
</details>

In [ ]:
# YOUR CODE HERE

# get the path of the image at index 53
path_53 = dataset_paths[53]

# open the image 
from PIL import Image
import matplotlib.pyplot as plt

with Image.open(path_53) as im:
    plt.imshow(im)

❓ **What is the shape of the image you displayed above `img_shape`?  How many dimensions `img_dim` does it have ?**

In [ ]:
# YOUR CODE HERE

# shape of the image 
image = Image.open(path_53)
# size is given (width, height)
image.size

In [ ]:
import numpy as np

# Set seed so that results are reproduceable
np.random.seed(42)

np_image = np.array(im)
# (row, col, depth)
np_image.shape

In [ ]:
# as it is a colourful image -> 3 dimensions for RGB
img_dim = 3

❓ **What was in the image above?**

In [ ]:
img_shape = np_image.shape
img_dim = img_dim

#is_portrait = True
is_portrait = False

is_colored_image = True
#is_colored_image = False

## 2. Look for duplicates

👉 Let's check if we have similar images in the dataset.

In [ ]:
from collections import defaultdict
import os
import sys
from multiprocessing import cpu_count, Pool
from pathlib import Path
import tqdm
from typing import Dict, List, Optional

from scipy.fftpack import dct

class Hasher:
    def __init__(self, verbose: bool = False) -> None:
        self.target_size = (32, 32)
        self.__coefficient_extract = (8, 8)
        self.verbose = verbose
        self.query_results_map = None

    @staticmethod
    def hamming_distance(hash1: str, hash2: str) -> float:
        hash1_bin = bin(int(hash1, 16))[2:].zfill(64)
        hash2_bin = bin(int(hash2, 16))[2:].zfill(64)
        return np.sum([i != j for i, j in zip(hash1_bin, hash2_bin)])
    
    def encode_image(self, image_file=None) -> str:
        try:
            if image_file and os.path.exists(image_file):
                image_file = Path(image_file)
                img = Image.open(image_file)
    
                if img.format not in ['JPEG', 'PNG', 'BMP', 'MPO', 'PPM', 'TIFF', 'GIF']:
                    return None
    
                if img.mode != 'RGB':
                    # convert to RGBA first to avoid warning
                    # we ignore alpha channel if available
                    img = img.convert('RGBA').convert('RGB')
                    
                
                image_pp = np.array(img.resize(self.target_size, Image.ANTIALIAS).convert('L')).astype('uint8')
            else:
                raise ValueError
        except (ValueError, TypeError):
            raise ValueError('Please provide either image file path')

        return self._hash_func(image_pp) if isinstance(image_pp, np.ndarray) else None
    
    def encode_images(self, image_dir=None) -> Dict[str, str]:
        if not os.path.isdir(image_dir):
            raise ValueError('Please provide a valid directory path!')

        image_dir = Path(image_dir)

        files = [
            i.absolute() for i in image_dir.glob('*') if not i.name.startswith('.')
        ]  # ignore hidden files
        
        #pool = Pool(processes=cpu_count())
        #hashes = list(
        #    tqdm.tqdm(pool.imap(self.encode_image, files, 100), total=len(files), disable=not self.verbose)
        #)
        #pool.close()
        #pool.join()
        
        hashes = [self.encode_image(f) for f in files]
        
        hash_initial_dict = dict(zip([f.name for f in files], hashes))
        hash_dict = {
            k: v for k, v in hash_initial_dict.items() if v
        }  # To ignore None (returned if some probelm with image file)

        return hash_dict
    
    def _hash_func(self, image_array: np.ndarray) -> str:
        dct_coef = dct(dct(image_array, axis=0), axis=1)

        # retain top left 8 by 8 dct coefficients
        dct_reduced_coef = dct_coef[: self.__coefficient_extract[0], : self.__coefficient_extract[1]]

        # median of coefficients excluding the DC term (0th term)
        median_coef_val = np.median(np.ndarray.flatten(dct_reduced_coef)[1:])

        # return mask of all coefficients greater than mean of coefficients
        hash_mat = dct_reduced_coef >= median_coef_val
        
        return ''.join('%0.2x' % x for x in np.packbits(hash_mat))
    
    def find_duplicates(self, image_dir: str = None, encoding_map: List[str] = None, threshold: int = 10) -> Dict:
        if not encoding_map:
            encoding_map = self.encode_images(image_dir)
            
        result_map = defaultdict(list)

        for i, (ki, vi) in enumerate(encoding_map.items()):
            for j, (kj, vj) in enumerate(encoding_map.items()):
                if i < j:
                    dij = self.hamming_distance(vi, vj)
                    if dij <= threshold:
                        result_map[ki].append((kj, dij))
                        result_map[kj].append((ki, dij))
        
        self.query_results_map = {
            k: [i for i in sorted(v, key=lambda tup: tup[1], reverse=False)]
            for k, v in result_map.items()
        }
                
        return self.query_results_map
    
    def get_files_to_remove(self, duplicates: Dict[str, List] = None) -> List:
        files_to_remove = set()
        
        for k, v in duplicates.items():
            tmp = [i[0] for i in v]
            
            if k not in files_to_remove:
                files_to_remove.update(tmp)
                
        return list(files_to_remove)

In [ ]:
%%time
hasher = Hasher()
encodings = hasher.encode_images('paintings')

In [ ]:
duplicates = hasher.find_duplicates(encoding_map=encodings)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec

image_dir = 'paintings'

for original, image_list in duplicates.items():    
    n_images = len(image_list)
    ncols = 4  # fixed for a consistent layout
    nrows = int(np.ceil(n_images / ncols)) + 1
    fig = plt.Figure(figsize=(10, 14))

    gs = gridspec.GridSpec(nrows=nrows, ncols=ncols)
    ax = plt.subplot(gs[0, 1:3])  # Always plot the original image in the middle of top row
    ax.imshow(Image.open(f"{image_dir}/{original}"))
    ax.set_title(f"Original Image: {original}")
    ax.axis('off')

    for i in range(0, n_images):
        row_num = (i // ncols) + 1
        col_num = i % ncols

        ax = plt.subplot(gs[row_num, col_num])
        ax.imshow(Image.open(f"{image_dir}/{image_list[i][0]}"))
        title = ' '.join([image_list[i][0], f"({image_list[i][1]})"])
        ax.set_title(title, fontsize=9)
        ax.axis('off')

    plt.show()
    plt.close()

In [ ]:
hasher.get_files_to_remove(duplicates)

## 3. Processing

❓ **Store all images from the dataset folder in a list of numpy arrays called `dataset_images`**

- It can take a while
- If the dataset is too big to fit in memory, just take the first half (or quarter) of all pictures

In [ ]:
# YOUR CODE HERE

dataset_images = []

for img_name in dataset_paths:
    image = Image.open(img_name)
    dataset_images.append(np.array(image))
    
print(type(dataset_images))

In [ ]:
len(dataset_images), dataset_images[0].shape

### 3.1 Reshape, Resize, Rescale

Let's simplify our dataset and convert it to a single numpy array

❓ **First, check if that all the images in the dataset have the same number of dimensions**.
- What do you notice?
- How do you explain it? 

In [ ]:
dim1, dim2, dim3 = [], [], []
for img in dataset_images:
    shap = img.shape
    dim1.append(shap[0])
    dim2.append(shap[1])
    if len(shap) > 2:
        dim3.append(shap[2])
        
if len(dim3) == len(dim2):
    print("All images are RGB")
else:
    print("Some images are B&W")

👉 We convert for you all black & white images into 3-colored ones by duplicating the image on three channels, so as to have only 3D arrays

In [ ]:
from tqdm import tqdm

dataset_images = [x if x.ndim == 3 else np.repeat(x[:,:,None], 3, axis=2) for x in tqdm(dataset_images)]

# Print the set of the number of dimensions of all arrays (it should contain only the value 3)
set([x.ndim for x in dataset_images])

❓ **What about their shape now ?**
- Do they all have the same width/heights ? If not:
- Resize the images (120 pixels height and 100 pixels width) in the dataset, using `tensorflow.image.resize` function.
- Now that they all have the same shape, store them as a numpy array `dataset_resized`.
- This array should thus be of size $(n_{images}, 120, 100, 3)$

In [ ]:
# YOUR CODE HERE

if len(set(dim1)) == 1 and len(set(dim2)) == 1:
    print("Images dont have all the same size")
else:
    print("Images dont have the same size")

In [ ]:
print("[height]", "max:", max(dim1), "min:", min(dim1))
print("[width] ", "max:", max(dim2), "min:", min(dim2))

In [ ]:
# resizing 
from tensorflow.image import resize, rgb_to_grayscale, image_gradients

In [ ]:
%%time 
# resizing all

resized_img = []

for img in dataset_images:
    resized_img.append(resize(img, (120, 100), antialias=True).numpy())

In [ ]:
dataset_resized = np.array(resized_img)

In [ ]:
dataset_resized.shape

❓ **Rescale the data of each image between $0$ and $1$**
- Save your resulting list as `dataset_scaled`

In [ ]:
# YOUR CODE HERE
print(np.max(dataset_resized), np.min(dataset_resized))

In [ ]:
# Let's divide by 255 

dataset_scaled = dataset_resized / 255

In [ ]:
# Let's check the max and mean:
print(np.min(dataset_scaled), np.max(dataset_scaled))

### 3.2 Create (X, y) sets

👉 Now, we'll add for you some **random noise** to our images to simulate noise (that our model will try to remove later)

In [ ]:
NOISE_LEVEL = 0.2

# Compute Gaussian noise
mean = 0
var = 0.1
sigma = var**0.5
noise = NOISE_LEVEL*np.random.normal(mean, sigma, size=dataset_scaled.shape)

# There are other types of noise: poisson, speckle, salt_n_pepper

print(type(noise), noise.shape, noise.dtype)
dataset_noisy = dataset_scaled + noise

# Images data can now be in [-0.2, 1.2]
print(f"Before clipping: [{np.min(dataset_noisy)}, {np.max(dataset_noisy)}]")

dataset_noisy = np.clip(dataset_noisy, 0, 1)
print(f"After clipping: [{np.min(dataset_noisy), np.max(dataset_noisy)}]")

dataset_noisy.shape

❓ **Plot a noisy image below to visualize the noise and compare it with the normal one**

In [ ]:
# YOUR CODE HERE

for i, (scaled_image, noisy_image) in enumerate(zip(dataset_scaled, dataset_noisy)):
    _, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 10))
    ax1.imshow(scaled_image)
    ax1.axis('off')
    ax2.imshow(noisy_image)
    ax2.axis('off')
    plt.show()
    
    if i > 5:
        break

❓ **Create your `(X_train, Y_train)`, `(X_test, Y_test)` training set for your problem**

- Remember you are trying to use "noisy" pictures in order to predict the "normal" ones.
- Keeping about `20%` of randomly sampled data as test set

In [ ]:
# YOUR CODE HERE

from sklearn.model_selection import train_test_split

X = dataset_noisy
y = dataset_scaled

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape, Y_train.shape, X_train[:,:,:,0].std(), Y_train[:,:,:,0].std())
# Y_train[0]

## 4. Convolutional Neural Network

A commonly used neural network architecture for image denoising is the __AutoEncoder__.

<img src='https://github.com/lewagon/data-images/blob/master/DL/autoencoder.png?raw=true'>

Its goal is to learn a compact representation of your data to reconstruct them as precisely as possible.  
The loss for such model must incentivize it to have __an output as close to the input as possible__.

For this challenge, __you will only be asked to code the Encoder part of the network__, since building a Decoder leverages layers architectures you are not familiar with (yet).

👉 Run this code below if you haven't managed to build your own (X,Y) training sets. This will load them as solution

```python
! curl https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/data_painting_solution.pickle > data_painting_solution.pickle

import pickle
with open("data_painting_solution.pickle", "rb") as file:
    (X_train, Y_train, X_test, Y_test) = pickle.load(file)
    
! rm data_painting_solution.pickle
```

In [ ]:
! curl https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/data_painting_solution.pickle > data_painting_solution.pickle

import pickle
with open("data_painting_solution.pickle", "rb") as file:
    (X_train_ref, Y_train_ref, X_test_ref, Y_test_ref) = pickle.load(file)

! rm data_painting_solution.pickle

In [ ]:
to_display = [[50, 50], [62, 110], [228, 207], [187, 322], [238, 8]]

for i, pair in enumerate(to_display):
    _, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(1, 5, figsize=(20, 20))
    ax1.imshow(Y_train[pair[0]])
    ax1.axis('off')
    ax2.imshow(Y_train_ref[pair[1]])
    ax2.axis('off')
    ax3.imshow(Y_train[pair[0]]-Y_train_ref[pair[1]])
    ax3.axis('off')
    ax4.imshow(X_train[pair[0]])
    ax4.axis('off')
    ax5.imshow(X_train_ref[pair[1]])
    ax5.axis('off')
    print(f"{np.sum(Y_train[pair[0]]-Y_train_ref[pair[1]])} - {X_train[pair[0],:, :, 0].std()} vs {X_train_ref[pair[1],:, :, 0].std()}")
    plt.show()

### 4.1 Architecture

👉 Run the cell below that defines the decoder

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, losses, Sequential

In [ ]:
# We choose to compress images into a latent_dimension of size 6000
latent_dimensions = 6000

# We build a decoder that takes 1D-vectors of size 6000 to reconstruct images of shape (120,100,3)
decoder = Sequential(name='decoder')
decoder.add(layers.Reshape((30, 25, 8), input_dim=latent_dimensions))
decoder.add(layers.Conv2DTranspose(filters=16, kernel_size=3, strides=2, padding="same", activation="relu"))
decoder.add(layers.Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding="same", activation="relu"))
decoder.add(layers.Conv2D(filters=3, kernel_size=3, padding="same", activation="sigmoid"))
decoder.summary()

❓ **Now, build the `encoder` that plugs correctly with the decoder defined above**. Make sure that:
- The output of your `encoder` is the same shape as the input of the `decoder`
- Use a convolutional neural network architecture without transfer learning
- Keep it simple
- Print model summary

In [ ]:
# CODE HERE YOUR ENCODER ARCHITECTURE AND PRINT IT'S MODEL SUMMARY

from tensorflow.keras import Sequential

def build_encoder(latent_dimension):
    encoder = Sequential(name="encoder")
    encoder.add(layers.InputLayer(input_shape=(120, 100, 3)))
    encoder.add(layers.Conv2D(32, 5, strides=2, padding='same', activation=layers.LeakyReLU(0.1)))
    encoder.add(layers.Dropout(0.2))
    encoder.add(layers.Conv2D(8, 3, strides=2, padding='same', activation=layers.LeakyReLU(0.1)))
    encoder.add(layers.Dropout(0.3))
    encoder.add(layers.Flatten())
    # encoder.add(Dense(latent_dimension, activation='sigmoid'))
    
    return encoder

In [ ]:
encoder = build_encoder(6000)
encoder.summary()

👉 **Test your encoder below**

In [ ]:
# HERE WE BUILD THE AUTO-ENCODER (ENCODER + DECODER) FOR YOU. IT SHOULD PRINT A NICE SUMMARY
from tensorflow.keras.models import Model

x = layers.Input(shape=(120, 100, 3))
autoencoder = Model(x, decoder(encoder(x)), name="autoencoder")
autoencoder.summary()

### 4.2 Training

❓ **Before training the autoencoder, evaluate your baseline score**
- We will use the mean absolute error in this challenge
- Compute the baseline score on your test set in the "stupid" case where you don't manage to de-noise anything at all.
- Store the result under `score_baseline`

In [ ]:
# YOUR CODE HERE
import tensorflow as tf

# Baseline: we predict the noisy image

class Baseline(tf.keras.Model):
    def __init__(self):
        super(Baseline, self).__init__()
        
    def call(self, x):
        return x

baseline = Baseline()

baseline.compile(loss='binary_crossentropy', optimizer='adam')

# An other way to do it !
# print(tf.keras.metrics.mean_absolute_error(Y_test, X_test).numpy().mean())

score_baseline = baseline.evaluate(X_test, Y_test)
score_baseline

❓ Now, **train your autoencoder**

- Use an appropriate loss
- Adapt the learning rate of your optimizer if convergence is too slow/fast
- Make sure your model does not overfit with appropriate control techniques

💡 You will not be judged by the computing power of your computer, you can reach decent performance in less than 5 minutes of training without GPUs.

In [ ]:
def compile_autoencoder(autoencoder):
    autoencoder.compile(loss='mae', optimizer=tf.optimizers.Adam(0.0005))

In [ ]:
# YOUR CODE 

# Attention no metric is define 
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=25, verbose=1, restore_best_weights=True)

compile_autoencoder(autoencoder)

history_denoising = autoencoder.fit(X_train, Y_train, 
                                    validation_split=0.2, epochs=1000,
                                    shuffle = True,
                                    batch_size=32, callbacks=[es])

❓ **Plot your training and validation loss at each epoch using the cell below**

In [ ]:
# Plot below your train/val loss history
# YOUR CODE HERE

def plot_history(history, title='', axs=None, exp_name=""):
    f = plt.figure(figsize=(10,7))
    f.add_subplot()

    #Adding Subplot
    plt.plot(history.epoch, history.history['loss'], label = "loss") # Loss curve for training set
    plt.plot(history.epoch, history.history['val_loss'], label = "val_loss") # Loss curve for validation set

    plt.title("Loss Curve",fontsize=18)
    plt.xlabel("Epochs",fontsize=15)
    plt.ylabel("Loss",fontsize=15)
    plt.grid(alpha=0.3)
    plt.legend()
    #plt.savefig("Loss_curve.png")
    return f

plot_history(history_denoising)


# Run also this code to save figure as jpg in path below (it's your job to ensure it works)
fig = plt.gcf()
plt.savefig("history.png")

❓ **Evaluate your performances on test set**
- Compute your de-noised test set `Y_pred` 
- Store your test score as `score_test`
- Plot a de-noised image from your test set and compare it with the original and noisy one using the cell below

In [ ]:
# YOUR CODE HERE
score_test = autoencoder.evaluate(X_test, Y_test)

In [ ]:
# RUN THIS CELL TO CHECK YOUR RESULTS

Y_pred = autoencoder.predict(X_test)

l = np.arange(len(X_test))
np.random.shuffle(l)

for i in l[:5]:
    fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(10,5))
    ax1.imshow(Y_test[i])
    ax1.set_title("Clean image.")
    ax1.axis('off')

    ax2.imshow(X_test[i])
    ax2.set_title("Noisy image.")
    ax2.axis('off')

    ax3.imshow(Y_pred[i])
    ax3.set_title("Prediction.")
    ax3.axis('off')

# Run this to save your results for correction
plt.savefig('image_denoised.png')

## 5. Other stuff (Data Augmentation)

### 5.1 Use dataset from numpy array

In [ ]:
#ds = tf.data.Dataset.from_tensor_slices(())

### 5.2 Use dataset from directory

In [ ]:
list_files = glob.glob('paintings/*.jpg')

n_full = len(list_files)

n_train = int(0.7*n_full)
n_test = int(0.15*n_full)
n_val = n_full - n_train - n_test

print(f"n_train: {n_train} - n_val: {n_val} - n_test: {n_test}")

ds_full = tf.data.Dataset.from_tensor_slices((list_files, list_files)).shuffle(n_full, reshuffle_each_iteration=False)
ds_train = ds_full.take(n_train)
ds_test = ds_full.skip(n_train)
ds_val = ds_test.skip(n_val)
ds_test = ds_test.take(n_test)

def load(file_path, add_noise=False):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, size=(120, 100), antialias=True)
    if add_noise:
        img += 0.2*tf.random.normal((120, 100, 3), stddev=0.316, dtype=tf.dtypes.float32)
    img = tf.clip_by_value(img, 0, 1)
    
    return img

def process_image(file_path1, file_path2):
    img = tf.stack([load(file_path1), load(file_path2, add_noise=True)])
    choice = tf.random.uniform((), minval=0, maxval=1)
    if choice < 0.5:
        img = tf.image.random_hue(img, max_delta=.5)
    choice = tf.random.uniform((), minval=0, maxval=1)
    if choice < 0.5:
        img =  tf.image.flip_left_right(img)
    choice = tf.random.uniform((), minval=0, maxval=1)
    if choice < 0.5:
        img =  tf.image.flip_up_down(img)

    return img[0], img[1]

ds_train = ds_train.map(lambda x, y: process_image(x, y), num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(n_train).batch(64).prefetch(2)
ds_val = ds_val.map(lambda x, y: process_image(x, y), num_parallel_calls=tf.data.experimental.AUTOTUNE).shuffle(n_val).batch(64)
ds_test = ds_test.map(lambda x, y: process_image(x, y), num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(64)

#AUTOTUNE = tf.data.AUTOTUNE

for (x, y) in ds_train.take(5):
    _, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    print(np.min(x[0, ...]), np.max(x[0, ...]), np.min(y[0, ...]), np.max(y[0, ...]))
    ax1.imshow(x[0, ...])
    ax1.axis('off')
    ax2.imshow(y[0, ...])
    ax2.axis('off')

    plt.show()

In [ ]:
latent_dim = 6000 

class Autoencoder(tf.keras.Model):
    def __init__(self, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim   
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=(120, 100, 3)),
            tf.keras.layers.Conv2D(32, 5, strides=2, padding='same', activation=layers.LeakyReLU(0.1)),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Conv2D(8, 3, strides=2, padding='same', activation=layers.LeakyReLU(0.1)),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Flatten(),
        ])
        self.decoder = tf.keras.Sequential([        
            tf.keras.layers.Reshape((30, 25, 8), input_dim=self.latent_dim),
            tf.keras.layers.Conv2DTranspose(filters=16, kernel_size=3, strides=2, padding='same', activation='relu'),
            tf.keras.layers.Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='same', activation='relu'),
            tf.keras.layers.Conv2D(filters=3, kernel_size=3, padding='same', activation='sigmoid')
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = Autoencoder(latent_dim)

In [ ]:
autoencoder.compile(optimizer='adam', loss=tf.keras.losses.MeanAbsoluteError())

In [ ]:
%%time
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=25, verbose=1, restore_best_weights=True)

# reduce learning rate when on a plateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", mode="min", verbose=1, patience=10, min_delta=0.0001, factor=0.2,)

history = autoencoder.fit(ds_train, epochs=1_000, shuffle=True, validation_data=ds_val, callbacks=[es, reduce_lr])

In [ ]:
plot_history(history);

In [ ]:
autoencoder.evaluate(ds_test)

In [ ]:
ds_test_iter = ds_test.make_one_shot_iterator()

i = 0
for (x, y) in ds_test_iter.get_next():
    _, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10, 5))

    ax1.imshow(y[0, ...])
    ax1.set_title("Clean image.")
    ax1.axis('off')

    ax2.imshow(x[0, ...])
    ax2.set_title("Noisy image.")
    ax2.axis('off')
    
    y_pred = autoencoder.predict(x)

    ax3.imshow(y_pred[i])
    ax3.set_title("Prediction.")
    ax3.axis('off')
    
    

    i = i + 1
    
    plt.show()